In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
import numpy as np


In [8]:

(train_data, test_data), info = tfds.load('mnist', as_supervised=True, split=['train', 'test'], with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\warad\tensorflow_datasets\mnist\incomplete.KI0IFD_3.0.1\mnist-train.tfrecord*...:   0%|    …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\warad\tensorflow_datasets\mnist\incomplete.KI0IFD_3.0.1\mnist-test.tfrecord*...:   0%|     …

Dataset mnist downloaded and prepared to C:\Users\warad\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [9]:
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, (32, 32))
    return image, label

In [10]:
train_data = train_data.map(preprocess_image).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_image).batch(64).prefetch(tf.data.AUTOTUNE)

In [11]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 38s 1us/step


In [12]:
for layer in vgg_model.layers:
    layer.trainable = False

In [13]:
custom_classifier = models.Sequential([
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

In [14]:
model = models.Sequential([
    vgg_model,
    custom_classifier
])

In [15]:

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, epochs=5, validation_data=test_data)

Epoch 1/5
 10/938 ━━━━━━━━━━━━━━━━━━━━ 3:38 236ms/step - accuracy: 0.1191 - loss: 2.3797

In [ ]:
loss, acc = model.evaluate(test_data)
print("Test Loss:", loss)
print("Test Accuracy:", acc)

pred = model.predict(test_data)

In [ ]:
labels = [str(i) for i in range(10)]

In [ ]:
for image, label in test_data.take(1):
    idx = 0
    plt.imshow(image[idx], cmap='gray')
    plt.title(f"Predicted: {labels[np.argmax(pred[idx])]} - True: {labels[label[idx]]}")
    plt.show()

print("True label:", labels[label[idx]])